In [1]:
import numpy
from pybit.unified_trading import HTTP
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from models.TS.GRUPipeline import GRUNetwork
import time
from torch import optim
from torch import nn
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

In [2]:
SYMBOL = 'SOLUSDT'
API_KEY = 'Z8QsHgyTpwinr9K2Ya'
API_SECRET = '5QfjbglBfFK5A8RqGvcvdw8hCF4Yjv5sByOn' # март

In [3]:
cl = HTTP(
    
)

In [4]:
#butch_size = 5
START_TIMESTAMP = '1711929617184'
time_series = 1000 #  количество изначальных временных рядов в разные промежутки времени, с количеством измерений t_s_l
time_series_length = 10 #  количество сэмплов из ts, на которые мы делим временной ряд
time_series_sample = 20 #  количество сделок в одном временном ряду

time_series_real_l = time_series_length + time_series_sample - 1
time_stamps = [START_TIMESTAMP]

train_data = np.ndarray(shape=(time_series, time_series_length, time_series_sample))
volumes = np.ndarray(shape=time_series_real_l*time_series)
all_price = np.ndarray(shape=time_series_real_l*time_series)

for i in range(time_series):
    kline = cl.get_kline(
        category="spot",
        symbol=SYMBOL,
        start=time_stamps[-1],
        interval=1,
        limit = time_series_real_l
    )
    if kline.get('result', []).get('list', [])[-1][0] < time_stamps[-1]:
        time_stamps[-1] = kline.get('result', []).get('list', [])[-2][0]
        time.sleep(0.05)
        kline = cl.get_kline(
            category="spot",
            symbol=SYMBOL,
            start=time_stamps[-1],
            interval=1,
            limit = time_series_real_l
        )
    if i % 120 == 0:
        time.sleep(5)
    time_stamps.append(str( int(kline.get('result', []).get('list', [])[0][0]) + 10000))
    
    volumes[i*time_series_real_l:(i+1)*time_series_real_l] = np.array([i[5] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)
    all_price[i*time_series_real_l:(i+1)*time_series_real_l] = np.array([i[4] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)
    
    prices = np.array([i[4] for i in kline.get('result', []).get('list', [])[::-1]], dtype=float)         
    train_data[i] = np.array(
        [prices[i:time_series_sample+i] for i in range(time_series_real_l-time_series_sample+1)], dtype=float
    ).reshape(time_series_real_l-time_series_sample+1, time_series_sample)     

In [5]:
print(train_data.shape)

(1000, 10, 20)


In [11]:
train_size = int(time_series * 0.8)
prediction_length = 10
idxs = np.random.choice((time_series-1), size=train_size)
x_train, y_train, = torch.tensor(train_data[idxs]), torch.tensor(train_data[idxs+1][:, 0, :prediction_length])
print(x_train.shape, y_train.shape)

torch.Size([800, 10, 20]) torch.Size([800, 10])


In [12]:
synthetic_data_size = 2000
#synthetic_data_x = np.ndarray(shape=(synthetic_data_size, time_series_length, time_series_sample))
#synthetic_data_y = np.ndarray(shape=(synthetic_data_size, prediction_length))
synthetic_data_x = []
synthetic_data_y = []
for i in range(2000):
    idx = int(np.random.rand()*(train_size-2)) #  можно взять предпоследний элемент, потому что по следующему эл-ту будет идти ошибка
    size_s = min(time_series_length-1, max(2, int(np.random.normal(time_series_length/2, time_series_length/4))))
    samples_idx = np.random.choice(time_series_length-1, size = size_s, replace=True)
    samples_idx.sort()
    samples_idx = np.append(samples_idx, time_series_length-1)
    el_x = x_train[idx][samples_idx]
    el_y = x_train[idx+1][0, :prediction_length]
    synthetic_data_x.append(el_x.clone().detach().requires_grad_(True))
    synthetic_data_y.append(el_y.clone().detach().requires_grad_(True))
synthetic_data_x = pad_sequence(synthetic_data_x, batch_first=True)
synthetic_data_y = torch.stack(synthetic_data_y)
print(synthetic_data_x.shape, synthetic_data_y.shape)

torch.Size([2000, 10, 20]) torch.Size([2000, 10])


In [15]:
torch.save(torch.concat((x_train, synthetic_data_x), dim=0), f'train_x_{SYMBOL}.pt')
torch.save(torch.concat((y_train, synthetic_data_y), dim=0), f'train_y_{SYMBOL}.pt')
dataset = TensorDataset(torch.concat((x_train, synthetic_data_x), dim=0), torch.concat((y_train, synthetic_data_y), dim=0))
batch_size = 5
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [16]:
gru = GRUNetwork(time_series_sample, 50, 2, prediction_length, 1)

In [10]:
criterion = nn.MSELoss()
optimizer = optim.Adam(gru.parameters(), lr=0.01)

# Обучение модели
num_epochs = 1000
x_train = x_train.view(x_train.size(0), x_train.size(1), -1)  # Преобразование для приемлемого формата
y_train = y_train.view(y_train.size(0), -1)  # Преобразование для целевых значений

for epoch in range(num_epochs):
    for i, (batch_x, batch_y) in enumerate(dataloader):
        gru.train()
        outputs = gru(batch_x)
        optimizer.zero_grad()
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

ValueError: Неверная форма IMFs: torch.Size([20, 3])
Изначальные данные - tensor([[135.6900, 135.8400, 135.6200, 135.5800, 135.6700, 136.1900, 136.6300,
         136.3300, 136.3600, 136.3300, 136.6100, 136.6400, 136.1700, 136.3300,
         136.5000, 136.4000, 136.3100, 136.2600, 135.9700, 135.9100],
        [135.6700, 136.1900, 136.6300, 136.3300, 136.3600, 136.3300, 136.6100,
         136.6400, 136.1700, 136.3300, 136.5000, 136.4000, 136.3100, 136.2600,
         135.9700, 135.9100, 135.9500, 135.9600, 135.9600, 135.6100],
        [136.1900, 136.6300, 136.3300, 136.3600, 136.3300, 136.6100, 136.6400,
         136.1700, 136.3300, 136.5000, 136.4000, 136.3100, 136.2600, 135.9700,
         135.9100, 135.9500, 135.9600, 135.9600, 135.6100, 135.3500],
        [136.6300, 136.3300, 136.3600, 136.3300, 136.6100, 136.6400, 136.1700,
         136.3300, 136.5000, 136.4000, 136.3100, 136.2600, 135.9700, 135.9100,
         135.9500, 135.9600, 135.9600, 135.6100, 135.3500, 135.4900],
        [136.3